---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize, scale

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
#     for x in P1_Graphs:
#         print(nx.average_clustering(x), nx.average_shortest_path_length(x))
#         print()
#         degrees = x.degree()
#         degree_values = sorted(set(degrees.values()))
#         histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(x)) for i in degree_values]
#         plt.figure(figsize=(10, 10))
#         plt.plot(degree_values,histogram, 'o') 
#         plt.xlabel('Degree') 
#         plt.ylabel('Fraction of Nodes') 
#         plt.xscale('log')
#         plt.yscale('log') 
#         plt.show()
#         print()
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    # Your Code Here
    G = nx.read_gpickle('email_prediction.txt')
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    
    cols = ['clustering', 'degree', 'closeness', 'betweenness', 'page_rank']
    df[cols] = scale(df[cols])

    mask = df['ManagementSalary'].isnull()
    df_pred = df[mask]
    df_main = df[~mask]
    
    y_main = df_main['ManagementSalary'].astype(bool)
    df_main = df_main[cols]
    df_pred = df_pred[cols]
    
    df_train, df_test, y_train, y_test = train_test_split(df_main, y_main, test_size=0.25)
    model = ExtraTreesClassifier(n_estimators=25, max_features=None, random_state=23)
    model.fit(df_train, y_train)
    test_pred, test_prob = model.predict(df_test), model.predict_proba(df_test)[:, 1]
    
    prob = model.predict_proba(df_pred)[:, 1]
    return pd.Series(prob, index=df_pred.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [8]:
def new_connections_predictions():
    # Your Code Here
    df = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    G = nx.read_gpickle('email_prediction.txt')
    
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['Common Neighbors'] = df.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
    df['jacard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['community common neighbors'] = (
        [i[2] for i in nx.cn_soundarajan_hopcroft(G, community='Department', ebunch=df.index)])
    df['community resource allocation'] = (
        [i[2] for i in nx.ra_index_soundarajan_hopcroft(G, community='Department', ebunch=df.index)])
    
    mask = df['Future Connection'].isnull()
    df_pred = df[mask]
    df_main = df[~mask]
    
    cols = [
        'preferential attachment', 
        'Common Neighbors',
        'jacard', 
        'community common neighbors',
        'community resource allocation'
    ]
    
    y_main = df_main['Future Connection'].astype(bool)
    df_main = df_main[cols]
    df_pred = df_pred[cols]
    
    model = LogisticRegression(random_state=20, solver='lbfgs')
    model.fit(df_main, y_main)
    prob = model.predict_proba(df_pred)[:, 1]
    return pd.Series(prob, index=df_pred.index)
